<a href="https://colab.research.google.com/github/syauqyideas/Colab_Runner_FluidX3D/blob/main/FluidX3D_Colab_Plane_External_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong>Copy FluidX3d Directory and make Project File

In [1]:
!pip install IPython
!pip install k3d
!pip install numpy-stl numpy matplotlib trimesh
!jupyter nbextension install --py --user k3d
!jupyter nbextension enable --py --user k3d
import k3d
!pip install "pyvista[jupyter]"
from google.colab import output
output.enable_custom_widget_manager()

import os

import IPython #at some point, i just bored to see the dirty output
from IPython.display import clear_output




k3d.switch_to_text_protocol()
!git clone https://github.com/syauqyideas/Colab_Runner_FluidX3D.git
!mkdir FluidX3D
%cd FluidX3D
!sudo apt install build-essential
!pip install numpy-stl
!wget https://github.com/ProjectPhysX/FluidX3D/archive/refs/heads/master.zip
!unzip master.zip
clear_output()
!mkdir -p /content/FluidX3D/FluidX3D-master/stl
#!cp /content/Colab_Runner_FluidX3D/concord_cut_large.stl /content/FluidX3D/FluidX3D-master/stl/
%cd FluidX3D-master

cp: cannot create regular file '/content/FluidX3D/FluidX3D-master/stl/': Not a directory
/content/FluidX3D/FluidX3D-master


<h2>Install OpenCL Driver and reinstall nvidia-driver

In [ ]:
# doing this line in internal xterm to prevent error
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm

#insert this code :
!sudo apt update && sudo apt upgrade -y
!sudo apt install -y g++ git make ocl-icd-libopencl1 ocl-icd-opencl-dev #nvidia-driver-550
clear_output() #you can take away this

In [ ]:
!apt install nvidia-driver-555
!sudo apt update && sudo apt upgrade


clear_output() #you can take away this

!nvcc --version #it is just check cuda versi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvcc --version

<h1><strong>Case Setup</h1>

<h2><strong>Physical Domain and STL define

In [3]:
physical_speed = "120.0" #@param {"type":"string","placeholder":"meter/sec"}
physical_length = "15.0" #@param {"type":"string","placeholder":"meter"}
physical_width = "10.0" #@param {"type":"string","placeholder":"meter"}
maximum_simulation_time = "1.0" #@param {"type":"string"}
physical_density = "1.42" #@param {"type":"string"}
general_viscosity = '1.48E-5f' #@param  {"type":"string"}
import os
ratio_object_to_bounding_box = "0.56" #@param  {"type":"string"}
lbm_speed = "0.075" #@param  {"type":"string"}
stl_file_source = '/content/F18Coarse.stl' #@param {allow-input : true}
os.environ['stl_file_source'] = stl_file_source
stl_file_name = 'F18Coarse.stl' #@param {allow-input : true}


<h3><strong>Bounding Box Sizing Ratio (in X, Y, and Z)<h3>

In [ ]:
res_x = "1.0" #@param {"type":"string"}
res_y = "3.0" #@param {"type":"string"}
res_z = "1.0" #@param {"type":"string"}

<h3><strong>LBM Cell sizing (with maximum vram approach)<h3>

In [ ]:
approach = 'gpu ram' #@param ['cell sizing','gpu ram']
maximum_gpu_ram = 12000 #@param
vram_utilization = 12000 #@param

<h3><strong>Bounding Box Condition Initiation

In [ ]:
#X Axis
x_min = 'inflow/outflow' #@param ['solid','inflow/outflow']
if x_min == 'inflow/outflow' :
  x_0 = 'TYPE_E'
elif x_min == 'solid' :
  x_0 = 'TYPE_S'
x_max = 'inflow/outflow' #@param ['solid','inflow/outflow']
if x_max == 'inflow/outflow' :
  x_1 = 'TYPE_E'
elif x_max == 'solid' :
  x_1 = 'TYPE_S'

#Y Axis
y_min = 'inflow/outflow' #@param ['solid','inflow/outflow']
if y_min == 'inflow/outflow' :
  y_0 = 'TYPE_E'
elif y_min == 'solid' :
  y_0 = 'TYPE_S'
y_max = 'inflow/outflow' #@param ['solid','inflow/outflow']
if y_max == 'inflow/outflow' :
  y_1 = 'TYPE_E'
elif x_max == 'solid' :
  y_1 = 'TYPE_S'

#z Axis
z_min = 'inflow/outflow' #@param ['solid','inflow/outflow']
if z_min == 'inflow/outflow' :
  z_0 = 'TYPE_E'
elif z_min == 'solid' :
  z_0 = 'TYPE_S'

z_max = 'inflow/outflow' #@param ['solid','inflow/outflow']
if z_max == 'inflow/outflow' :
  z_1 = 'TYPE_E'
elif z_max == 'solid' :
  z_1 = 'TYPE_S'

<h3><strong>Flow Orientation

In [ ]:
flow_orientation = 'y' #@param ['x','y','z']

<h2><strong>STL File Verication

In [17]:
# @title
import pyvista as pv
import os

# --- 2. Atur Backend PyVista untuk Colab ---
# GANTI 'panel' dengan 'trame'
print("Mengatur backend PyVista ke 'html'...")
pv.set_jupyter_backend('html')

# --- 3. Ambil file dari form Colab ---
# Pastikan sel form sudah dijalankan
file_path = stl_file_source

print(f"Mencoba memuat: {file_path} dengan PyVista...")

try:
    # --- 4. Muat Mesh ---
    mesh = pv.read(file_path)
    print(f"✅ PyVista berhasil memuat mesh (Vertices: {mesh.n_points})")

    # --- 5. Siapkan Plotter ---
    plotter = pv.Plotter(notebook=True, window_size=[1200, 600])

    # --- 6. Tambahkan Objek ke Plot ---

    # Tambahkan mesh Anda
    plotter.add_mesh(
        mesh,
        color='lightgrey',
        show_edges=True,  # Tampilkan wireframe tipis
        edge_color='black'
    )

    # Tambahkan grid (yang Anda inginkan)
    plotter.show_grid()

    # Tambahkan sumbu orientasi
    plotter.add_axes()

    # --- 7. Tampilkan Plot Interaktif ---
    print("Menampilkan plot PyVista...")
    plotter.show()

except FileNotFoundError:
    print(f"❌ ERROR: File tidak ditemukan di '{file_path}'")
except Exception as e:
    print(f"❌ ERROR: PyVista gagal: {e}")

Mengatur backend PyVista ke 'html'...
Mencoba memuat: /content/F18Coarse.stl dengan PyVista...
✅ PyVista berhasil memuat mesh (Vertices: 5721)
Menampilkan plot PyVista...


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [23]:
# @title
import trimesh
import numpy as np

# Pastikan variabel ini masih terisi dari form Colab
file_path = stl_file_source

try:
    print(f"Mencoba memuat {file_path} dengan trimesh...")
    # 'force='mesh'' penting untuk memastikan kita mendapatkan mesh, bukan 'scene'
    mesh = trimesh.load(file_path, force='mesh')

    print(f"✅ Mesh trimesh dimuat (Vertices: {len(mesh.vertices)})")
    print("Memanggil mesh.show()... Ini akan membuka viewer inline.")

    # .show() akan membuka viewer interaktif langsung di sel Colab

except FileNotFoundError:
    print(f"❌ ERROR: File tidak ditemukan di '{file_path}'")
except Exception as e:
    print(f"❌ ERROR: Trimesh gagal memuat atau menampilkan mesh: {e}")

Mencoba memuat /content/F18Coarse.stl dengan trimesh...
✅ Mesh trimesh dimuat (Vertices: 5721)
Memanggil mesh.show()... Ini akan membuka viewer inline.


<h3><strong>STL Size Verification

In [13]:
# @title

import trimesh
import numpy as np

# --- 1. Tentukan Parameter ---
input_file = stl_file_source  # Ganti dengan nama file STL Anda

# --- 2. Muat Mesh dan Inspeksi ---
try:
    mesh = trimesh.load(input_file, force='mesh')
    print(f"--- Inspeksi Bounding Box untuk: {input_file} (trimesh) ---")

    # 'bounds' adalah array (2, 3) -> [[min_x, min_y, min_z], [max_x, max_y, max_z]]
    bounds = mesh.bounds
    mins = bounds[0]
    maxs = bounds[1]

    # 'extents' adalah properti bawaan untuk (max - min) pada tiap sumbu
    dimensions = mesh.extents

    print("\n--- Koordinat Minimum ---")
    print(f"Min X: {mins[0]:.4f}")
    print(f"Min Y: {mins[1]:.4f}")
    print(f"Min Z: {mins[2]:.4f}")

    print("\n--- Koordinat Maksimum ---")
    print(f"Max X: {maxs[0]:.4f}")
    print(f"Max Y: {maxs[1]:.4f}")
    print(f"Max Z: {maxs[2]:.4f}")

    print("\n--- Properti Dimensi (Panjang) ---")
    print(f"Panjang X (Extents): {dimensions[0]:.4f}")
    print(f"Panjang Y (Extents): {dimensions[1]:.4f}")
    print(f"Panjang Z (Extents): {dimensions[2]:.4f}")

except FileNotFoundError:
    print(f"ERROR: File '{input_file}' tidak ditemukan.")
except Exception as e:
    # Trimesh akan error jika file rusak atau format tidak didukung
    print(f"Terjadi kesalahan saat memuat/memproses mesh: {e}")

--- Inspeksi Bounding Box untuk: /content/F18Coarse.stl (trimesh) ---

--- Koordinat Minimum ---
Min X: -282.5570
Min Y: -525.4692
Min Z: -58.5894

--- Koordinat Maksimum ---
Max X: 282.5646
Max Y: 320.5240
Max Z: 138.2578

--- Properti Dimensi (Panjang) ---
Panjang X (Extents): 565.1216
Panjang Y (Extents): 845.9932
Panjang Z (Extents): 196.8471


<h3><strong>stl Scaler

In [16]:
scalex = 0.5 #@param
scaley = 0.5 #@param
scalez = 0.5 #@param

import trimesh
import numpy as np

# --- 1. Tentukan Parameter ---

# Nama file
input_file = stl_file_source  # Ganti dengan nama file STL Anda
output_file_trimesh = stl_file_source

# Faktor Skala:
# 1.0 = 100% (tidak ada perubahan)
# 2.0 = 200% (dua kali lebih besar)
# 0.5 = 50% (setengah ukuran)
scale_x = scalex
scale_y = scaley
scale_z = scalez  # Contoh: Membuat objek 2x lebih tinggi di sumbu Z

# --- 2. Muat Mesh ---

try:
    # Muat mesh menggunakan trimesh.load()
    mesh = trimesh.load(input_file, force='mesh')
    print(f"Berhasil memuat: {input_file} (Vertices: {len(mesh.vertices)})")

    # --- 3. Terapkan Penskalaan ---

    # Buat vektor skala. 'trimesh' menerima list atau array NumPy.
    scale_vector = [scale_x, scale_y, scale_z]

    # Terapkan transformasi skala ke mesh
    # Secara default, ini menskalakan dari titik asal (0, 0, 0)
    mesh.apply_scale(scale_vector)

    print(f"Menerapkan skala: {scale_vector}")

    # --- 4. Simpan (Export) Mesh ---

    # Gunakan .export() untuk menyimpan
    mesh.export(output_file_trimesh)
    print(f"Berhasil menyimpan mesh terskala ke: {output_file_trimesh}")

except FileNotFoundError:
    print(f"ERROR: File '{input_file}' tidak ditemukan.")
    print("Pastikan Anda mengunggah file STL Anda dan namanya sudah benar.")
except Exception as e:
    # Trimesh akan error jika file rusak atau format tidak didukung
    print(f"Terjadi kesalahan saat memuat/memproses mesh: {e}")

Berhasil memuat: /content/F18Coarse.stl (Vertices: 5721)
Menerapkan skala: [0.5, 0.5, 0.5]
Berhasil menyimpan mesh terskala ke: /content/F18Coarse.stl


<h3><strong>STL Translator (unit in meter)

In [ ]:
translatex = 0 #@param
translatey = 0 #@param
translatez = -100 #@param

import numpy as np
from stl import mesh

# --- 1. Tentukan Parameter ---

# Nama file
input_file = stl_file_source  # Ganti dengan nama file STL Anda
output_file = stl_file_source

# Jumlah pergeseran (translasi)
# Tentukan seberapa jauh Anda ingin menggeser objek di setiap sumbu.
# Nilai positif menggeser ke arah positif sumbu (misal, X+),
# nilai negatif menggeser ke arah negatif (misal, X-).
delta_x = translatex
delta_y = translatey
delta_z = translatez

# --- 2. Buat Vektor Translasi ---
translation_vector = np.array([delta_x, delta_y, delta_z], dtype=np.float32)
print(f"Akan menggeser objek dengan vektor: {translation_vector}\n")

# --- 3. Muat, Geser, dan Simpan STL ---

try:
    # Muat mesh dari file
    your_mesh = mesh.Mesh.from_file(input_file)
    print(f"Berhasil memuat: {input_file}")

    # 'your_mesh.vectors' adalah array (N, 3, 3)
    # [N segitiga, 3 vertex, 3 koordinat (x,y,z)]

    # Terapkan translasi (pergeseran):
    # Kita cukup menambahkan vektor translasi ke setiap vertex.
    # NumPy (broadcasting) akan menangani ini secara otomatis.
    your_mesh.vectors = your_mesh.vectors + translation_vector

    # Simpan mesh yang telah dimodifikasi
    your_mesh.save(output_file)
    print(f"Berhasil menyimpan mesh tergeser ke: {output_file}")

except FileNotFoundError:
    print(f"ERROR: File '{input_file}' tidak ditemukan.")
    print("Pastikan Anda mengunggah file STL Anda dan namanya sudah benar.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Akan menggeser objek dengan vektor: [   0.    0. -100.]

Berhasil memuat: /content/f15Coarse.stl
Berhasil menyimpan mesh tergeser ke: /content/f15Coarse.stl


<h3><strong>STL Automatic Centering

In [10]:
# @title
import trimesh
import numpy as np

# --- 1. Tentukan Parameter ---

# Nama file
input_file = stl_file_source  # Ganti dengan nama file STL Anda
output_file_trimesh = stl_file_source

# --- 2. Muat Mesh ---

try:
    # Muat mesh
    mesh = trimesh.load(input_file, force='mesh')
    print(f"Berhasil memuat: {input_file}")

    # --- 3. Dapatkan Pusat (Centroid) ---

    # Dapatkan centroid (pusat geometris) dari mesh.
    # Ini adalah posisi [x, y, z] dari pusat objek saat ini.
    current_center = mesh.centroid
    print(f"Pusat objek saat ini (centroid): {np.round(current_center, 2)}")

    # --- 4. Terapkan Translasi ---

    # Kita perlu memindahkan objek dengan vektor yang berlawanan
    # dari pusatnya saat ini untuk memindahkannya ke (0, 0, 0).
    translation_vector = -current_center

    # Terapkan pergeseran
    mesh.apply_translation(translation_vector)

    print(f"Objek dipindahkan ke (0,0,0) menggunakan vektor: {np.round(translation_vector, 2)}")

    # Verifikasi (opsional): Centroid baru seharusnya sangat dekat dengan 0
    new_center = mesh.centroid
    print(f"Pusat objek baru: {np.round(new_center, 5)}")

    # --- 5. Simpan (Export) Mesh ---

    mesh.export(output_file_trimesh)
    print(f"\nBerhasil menyimpan mesh terpusat ke: {output_file_trimesh}")

except FileNotFoundError:
    print(f"ERROR: File '{input_file}' tidak ditemukan.")
except Exception as e:
    print(f"Terjadi kesalahan saat memuat/memproses mesh: {e}")

Berhasil memuat: /content/F18Coarse.stl
Pusat objek saat ini (centroid): [0. 0. 0.]
Objek dipindahkan ke (0,0,0) menggunakan vektor: [-0. -0. -0.]
Pusat objek baru: [ 0. -0.  0.]

Berhasil menyimpan mesh terpusat ke: /content/F18Coarse.stl


<h3><strong>Stl Rotator (unit in degre)

In [8]:
rotationx = 90 #@param
rotationy = 0 #@param
rotationz = 0 #@param

# @title
import trimesh
import numpy as np

# --- 1. Tentukan Parameter ---

# Nama file
input_file = stl_file_source  # Ganti dengan nama file STL Anda
output_file_trimesh = stl_file_source

# Sudut rotasi dalam DERAJAT
rot_x_deg = rotationx
rot_y_deg = rotationy
rot_z_deg = rotationz

# --- 2. Muat Mesh ---

try:
    # Muat mesh menggunakan trimesh.load()
    # Kita tambahkan force='mesh' untuk memastikan hasilnya adalah objek Trimesh
    # (bukan 'Scene'), terutama jika file STL-nya kompleks.
    mesh = trimesh.load(input_file, force='mesh')

    print(f"Berhasil memuat: {input_file} (Vertices: {len(mesh.vertices)})")

    # --- 3. Buat Matriks Transformasi & Terapkan ---

    # trimesh.transformations memiliki helper untuk membuat matriks

    # 1. Buat matriks rotasi X
    R_x = trimesh.transformations.rotation_matrix(
        angle=np.deg2rad(rot_x_deg),
        direction=[1, 0, 0]  # Sumbu X
    )

    # 2. Buat matriks rotasi Y
    R_y = trimesh.transformations.rotation_matrix(
        angle=np.deg2rad(rot_y_deg),
        direction=[0, 1, 0]  # Sumbu Y
    )

    # 3. Buat matriks rotasi Z
    R_z = trimesh.transformations.rotation_matrix(
        angle=np.deg2rad(rot_z_deg),
        direction=[0, 0, 1]  # Sumbu Z
    )

    # Gabungkan matriks (urutan Z-Y-X)
    R_combined = trimesh.transformations.concatenate_matrices(R_z, R_y, R_x)

    # Terapkan transformasi ke mesh
    mesh.apply_transform(R_combined)

    # --- 4. Simpan (Export) Mesh ---

    # Gunakan .export() untuk menyimpan
    mesh.export(output_file_trimesh)
    print(f"Berhasil menyimpan mesh terotasi ke: {output_file_trimesh}")

except FileNotFoundError:
    print(f"ERROR: File '{input_file}' tidak ditemukan.")
    print("Pastikan Anda mengunggah file STL Anda dan namanya sudah benar.")
except Exception as e:
    # Trimesh akan error jika file rusak atau format tidak didukung
    print(f"Terjadi kesalahan saat memuat/memproses mesh: {e}")

Berhasil memuat: /content/F18Coarse.stl (Vertices: 5721)
Berhasil menyimpan mesh terotasi ke: /content/F18Coarse.stl


<h3><strong>Verify STL File

In [ ]:
# @title
!mkdir -p /content/FluidX3D/FluidX3D-master/stl/
!cp $stl_file_source /content/FluidX3D/FluidX3D-master/stl/

<h3><strong>Code Declaration and Export of setup.cpp

In [ ]:
# @title
%cd /content
import os


cpp_content = f"""
#include "setup.hpp"


void main_setup() {{ // Concorde; required extensions in defines.hpp: FP16S, EQUILIBRIUM_BOUNDARIES, SUBGRID, INTERACTIVE_GRAPHICS
    // ################################################################## define simulation box size, viscosity and volume force ###################################################################
    const uint3 lbm_N = resolution(float3({res_x}f, {res_y}f, {res_z}f), {vram_utilization}u); // input: simulation box aspect ratio and VRAM occupation in MB, output: grid resolution
    const float si_u = {physical_speed};
    const float si_length={physical_length}, si_width={physical_width};
    const float si_T = {maximum_simulation_time};
    const float si_nu={general_viscosity}, si_rho={physical_density};
    const float lbm_length = {ratio_object_to_bounding_box}f*(float)lbm_N.y;
    const float lbm_u = {lbm_speed}f;
    units.set_m_kg_s(lbm_length, lbm_u, 1.0f, si_length, si_u, si_rho);
    const float lbm_nu = units.nu(si_nu);
    const ulong lbm_T = units.t(si_T);
    print_info("Re = "+to_string(to_uint(units.si_Re(si_width, si_u, si_nu))));
    //print_info("speed = "+to_string(to_uint(units.si_u)));
    LBM lbm(lbm_N, 1u, 1u, 1u, lbm_nu);
    // ###################################################################################### define geometry ######################################################################################
    const float3 center = float3(lbm.center().x, lbm.center().y, lbm.center().z);
    const float3x3 rotation = float3x3(float3(0, 0, 1), radians(0.0f));
    lbm.voxelize_stl(get_exe_path()+"../stl/{stl_file_name}", center, rotation, lbm_length);
    const uint Nx=lbm.get_Nx(), Ny=lbm.get_Ny(), Nz=lbm.get_Nz(); parallel_for(lbm.get_N(), [&](ulong n) {{ uint x=0u, y=0u, z=0u; lbm.coordinates(n, x, y, z);
        if(lbm.flags[n]!=TYPE_S) lbm.u.{flow_orientation}[n] = lbm_u; //set speed on solid S
        if(z==0u) lbm.flags[n] = {z_0}; // TYPE_E FOR EQUILIBRIUM SPEED - TYPE_S FOR SOLID BOUNDARY
        if(z==Nz-1u) lbm.flags[n] = {z_1}; // TYPE_E FOR EQUILIBRIUM SPEED - TYPE_S FOR SOLID BOUNDARY
        if(y==0u) lbm.flags[n] = {y_0}; // TYPE_E FOR EQUILIBRIUM SPEED - TYPE_S FOR SOLID BOUNDARY
        if(y==Ny-1u) lbm.flags[n] = {y_1}; // TYPE_E FOR EQUILIBRIUM SPEED - TYPE_S FOR SOLID BOUNDARY
        if(x==0u) lbm.flags[n] = {x_0}; // TYPE_E FOR EQUILIBRIUM SPEED - TYPE_S FOR SOLID BOUNDARY
        if(x==Nx-1u) lbm.flags[n] = {x_1}; //
    }}); // ####################################################################### run simulation, export images and data ##########################################################################
    lbm.graphics.visualization_modes = VIS_FLAG_SURFACE|VIS_Q_CRITERION;
    lbm.run(0u, lbm_T); // initialize simulation
    lbm.write_status();
    while(lbm.get_t()<=lbm_T) {{ // main simulation loop
#if defined(GRAPHICS) && !defined(INTERACTIVE_GRAPHICS)
        if(lbm.graphics.next_frame(lbm_T, 10.0f)) {{
            lbm.graphics.set_camera_free(float3(0.491343f*(float)Nx, -0.882147f*(float)Ny, 0.564339f*(float)Nz), -78.0f, 6.0f, 22.0f);
            lbm.graphics.write_frame(get_exe_path()+"export/front/");
            lbm.graphics.set_camera_free(float3(1.133361f*(float)Nx, 1.407077f*(float)Ny, 1.684411f*(float)Nz), 72.0f, 12.0f, 20.0f);
            lbm.graphics.write_frame(get_exe_path()+"export/back/");
            lbm.graphics.set_camera_centered(0.0f, 0.0f, 25.0f, 1.648722f);
            lbm.graphics.write_frame(get_exe_path()+"export/side/");
            lbm.graphics.set_camera_centered(0.0f, 90.0f, 25.0f, 1.648722f);
            lbm.graphics.write_frame(get_exe_path()+"export/top/");
            lbm.graphics.set_camera_free(float3(0.269361f*(float)Nx, -0.179720f*(float)Ny, 0.304988f*(float)Nz), -56.0f, 31.6f, 100.0f);
            lbm.graphics.write_frame(get_exe_path()+"export/wing/");
            lbm.graphics.set_camera_free(float3(0.204399f*(float)Nx, 0.340055f*(float)Ny, 1.620902f*(float)Nz), 80.0f, 35.6f, 34.0f);
            lbm.graphics.write_frame(get_exe_path()+"export/follow/");
        }}
#endif // GRAPHICS && !INTERACTIVE_GRAPHICS
        lbm.run(1u, lbm_T); // run dt time steps
    }}
    lbm.write_status();
}}
"""

# --- 4. Tulis konten ke file .cpp ---
file_name = "setup.cpp"
try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(cpp_content)
    print(f"✅ File '{file_name}' telah berhasil dibuat.")

    # --- 5. (Opsional) Verifikasi isi file ---
    print(f"\n--- Isi dari {file_name} ---")
    with open(file_name, "r", encoding="utf-8") as f:
        print(f.read())

except Exception as e:
    print(f"Terjadi error saat menulis file: {e}")

clear_output()

<h2>Edit your Project first depending on your Need</h2>
<h3>Read documentation First on /content/fluidx1/FluidX3D-master/DOCUMENTATION.md

<h2>Run Project and Render

In [ ]:
!cp /content/setup.cpp /content/FluidX3D/FluidX3D-master/src/
!cp /content/Colab_Runner_FluidX3D/defines.hpp /content/FluidX3D/FluidX3D-master/src/
%cd /content/FluidX3D/FluidX3D-master/
!rm -r /content/FluidX3D/FluidX3D-master/bin/export/*
!./make.sh

/content/FluidX3D/FluidX3D-master
Info: Detected Operating System: Linux
Info: Compiling with 2 CPU cores.
g++ -c src/graphics.cpp -o temp/graphics.o -std=c++17 -pthread -O -Wno-comment 
g++ -c src/info.cpp -o temp/info.o -std=c++17 -pthread -O -Wno-comment -I./src/OpenCL/include
g++ -c src/lbm.cpp -o temp/lbm.o -std=c++17 -pthread -O -Wno-comment -I./src/OpenCL/include
g++ -c src/main.cpp -o temp/main.o -std=c++17 -pthread -O -Wno-comment -I./src/OpenCL/include
g++ -c src/setup.cpp -o temp/setup.o -std=c++17 -pthread -O -Wno-comment -I./src/OpenCL/include
g++ temp/*.o -o bin/FluidX3D -std=c++17 -pthread -O -Wno-comment -I./src/OpenCL/include -L./src/OpenCL/lib -lOpenCL  
.-----------------------------------------------------------------------------.
|                       ______________   ______________                       |
|                       \   ________  | |  ________   /                       |
|                        \  \       | | | |       /  /                        |

In [ ]:
!./bin/FluidX3D

<h2>Convert Image Sequence to mp4

In [ ]:
%cd /content
%cd /content/FluidX3D/FluidX3D-master/bin/export
#!ls -v image-*.png | awk '{print "file \047" $0 "\047"}' > imagelist.txt
#!head -n 20 imagelist.txt > imagelist_test.txt
#!ffmpeg -f concat -i imagelist_test.txt -framerate 25 -c:v libx264 -pix_fmt yuv420p output_test.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/back/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_back_.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/follow/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_follow.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/front/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_front.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/side/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_side.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/top/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_top.mp4
!ffmpeg -framerate 24 -pattern_type glob -i "/content/FluidX3D/FluidX3D-master/bin/export/wing/image-*.png" -c:v h264_nvenc -qp 18 -pix_fmt yuv420p output_gpu_q_wing.mp4
#!ffmpeg -f concat -safe 0 -i imagelist.txt -framerate 25 -c:v h264_nvenc -pix_fmt yuv420p output_gpu.mp4

<h2>Copy result file to your Drive

In [ ]:
from IPython.display import display, Javascript

# Function to create an alert
def create_alert(message):
    display(Javascript(f'alert("{message}")'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create an alert
create_alert("Start Copying")

!mkdir /content/drive/MyDrive/fluidx1output
!cp -r /content/fluidx1/FluidX3D-master/bin/export /content/drive/MyDrive/fluidx1output

In [ ]:
!cp /content/FluidX3D/FluidX3D-master/bin/export/output_gpu_q* /content/drive/MyDrive/OpenFoam/